# Analysis of monthly rewards receipts

## Introduction

In this notebook, we'll analyze user, receipt, and brand data to identify the top 5 brands by transaction count

## Goals

- Identify Top 5 brands
- Describe how that Top 5 changees over a monthly timeframe

## Library Loading

We begin by loading the required libraries to create Dataframes, injest JSON data, perform SQL queries, and translate dates for analysis

In [1]:
import pandas as pd
import json
import sqlite3 as sql
from datetime import datetime


In [2]:
#Create connection for sqlite3
conn = sql.connect('default.db')

## Data Loading and Preprocessing

To complete this analysis, we must ingest JSON data. In this case, JSON files were invalid and had to be adjusted with end of line updates to make them valid and ingestible.

In [3]:
users = pd.read_json(r"users1.json")
#Strip out extraneous characters around user ID
users['_id'] = users['_id'].astype(str)
users['_id'] = users['_id'].str.slice(10,-2)
#Convert JSON date to datetime
users['createdDate'] = users['createdDate'].astype(str)
users['createdDate'] = users['createdDate'].str.slice(10,-1)
users['createdDate'] = pd.to_datetime(users['createdDate'], unit='ms')
users['lastLogin'] = users['lastLogin'].astype(str)
users['lastLogin'] = users['lastLogin'].str.slice(10,-1)
users['lastLogin'] = pd.to_datetime(users['lastLogin'], unit='ms')

print(users)
#Save resulting table in SQL environment for queries
users.to_sql('users', conn,if_exists='replace')


                          _id  active             createdDate  \
0    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04.800   
1    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04.800   
2    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04.800   
3    5ff1e1eacfcf6c399c274ae6    True 2021-01-03 15:25:30.554   
4    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04.800   
..                        ...     ...                     ...   
490  54943462e4b07e684157a532    True 2014-12-19 14:21:22.381   
491  54943462e4b07e684157a532    True 2014-12-19 14:21:22.381   
492  54943462e4b07e684157a532    True 2014-12-19 14:21:22.381   
493  54943462e4b07e684157a532    True 2014-12-19 14:21:22.381   
494  54943462e4b07e684157a532    True 2014-12-19 14:21:22.381   

                  lastLogin         role signUpSource state  
0   2021-01-03 15:25:37.858     consumer        Email    WI  
1   2021-01-03 15:25:37.858     consumer        Email    WI  
2   2021-01-03 15:25:37.858     c

495

In [4]:
brands = []
with open('brands.json', 'r') as file:
    for line in file:
        brands.append(json.loads(line))

brands = pd.DataFrame(brands)
#Strip out extraneous characters around brand ID
brands['_id'] = brands['_id'].astype(str)
brands['_id'] = brands['_id'].str.slice(10,-2)
#Convert cpg to string for use in analysis
brands['cpg'] = brands['cpg'].astype(str)
print(brands)
#Save resulting table in SQL environment for queries
brands.to_sql('brands', conn,if_exists='replace')

                           _id       barcode            category  \
0     601ac115be37ce2ead437551  511111019862              Baking   
1     601c5460be37ce2ead43755f  511111519928           Beverages   
2     601ac142be37ce2ead43755d  511111819905              Baking   
3     601ac142be37ce2ead43755a  511111519874              Baking   
4     601ac142be37ce2ead43755e  511111319917      Candy & Sweets   
...                        ...           ...                 ...   
1162  5f77274dbe37ce6b592e90c0  511111116752              Baking   
1163  5dc1fca91dda2c0ad7da64ae  511111706328  Breakfast & Cereal   
1164  5f494c6e04db711dd8fe87e7  511111416173      Candy & Sweets   
1165  5a021611e4b00efe02b02a57  511111400608             Grocery   
1166  6026d757be37ce6369301468  511111019930              Baking   

          categoryCode                                                cpg  \
0               BAKING  {'$id': {'$oid': '601ac114be37ce2ead437550'}, ...   
1            BEVERAGES  {'$id

1167

Import and clean receipts table

In [5]:
#Import and cleans receipts
receipts = []
with open('receipts.json', 'r') as file:
    for line in file:
        receipts.append(json.loads(line))

receipts = pd.DataFrame(receipts)
#Strip out extraneous characters around receipt ID
receipts['_id'] = receipts['_id'].astype(str)
receipts['_id'] = receipts['_id'].str.slice(10,-2)
#Convert JSON date to datetime
receipts['createDate'] = receipts['createDate'].astype(str)
receipts['createDate'] = receipts['createDate'].str.slice(10,-1)
receipts['createDate'] = pd.to_datetime(receipts['createDate'], unit='ms')
receipts['dateScanned'] = receipts['dateScanned'].astype(str)
receipts['dateScanned'] = receipts['dateScanned'].str.slice(10,-1)
receipts['dateScanned'] = pd.to_datetime(receipts['dateScanned'], unit='ms')
receipts['finishedDate'] = receipts['finishedDate'].astype(str)
receipts['finishedDate'] = receipts['finishedDate'].str.slice(10,-1)
receipts['finishedDate'] = pd.to_datetime(receipts['finishedDate'], unit='ms')
receipts['modifyDate'] = receipts['modifyDate'].astype(str)
receipts['modifyDate'] = receipts['modifyDate'].str.slice(10,-1)
receipts['modifyDate'] = pd.to_datetime(receipts['modifyDate'], unit='ms')
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].astype(str)
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].str.slice(10,-1)
receipts['pointsAwardedDate'] = pd.to_datetime(receipts['pointsAwardedDate'], unit='ms')
receipts['purchaseDate'] = receipts['purchaseDate'].astype(str)
receipts['purchaseDate'] = receipts['purchaseDate'].str.slice(10,-1)
receipts['purchaseDate'] = pd.to_datetime(receipts['purchaseDate'], unit='ms')
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].astype(str)


print(receipts)
receipts.to_sql('receipts', conn,if_exists='replace')


                           _id  bonusPointsEarned  \
0     5ff1e1eb0a720f0523000575              500.0   
1     5ff1e1bb0a720f052300056b              150.0   
2     5ff1e1f10a720f052300057a                5.0   
3     5ff1e1ee0a7214ada100056f                5.0   
4     5ff1e1d20a7214ada1000561                5.0   
...                        ...                ...   
1114  603cc0630a720fde100003e6               25.0   
1115  603d0b710a720fde1000042a                NaN   
1116  603cf5290a720fde10000413                NaN   
1117  603ce7100a7217c72c000405               25.0   
1118  603c4fea0a7217c72c000389                NaN   

                                bonusPointsEarnedReason  \
0     Receipt number 2 completed, bonus point schedu...   
1     Receipt number 5 completed, bonus point schedu...   
2                            All-receipts receipt bonus   
3                            All-receipts receipt bonus   
4                            All-receipts receipt bonus   
...      

## Create primary data set ranking brands by timeframe

- Months defined as 30 day periods defined based on the last day of transaction data, 3/8/2021 to ensure consistency in the length of timeframes for analysis
- Focused on consumer transactions specifically

In [56]:
#Brand rankings by 30 day period, without transaction from Fetch staff

query = '''
 SELECT b.brandCode,
    CASE WHEN ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) <= 30 THEN 'Current_Month'
   WHEN ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) > 30 AND ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) <= 60 THEN 'Previous_Month'
    WHEN ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) > 60 AND ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) <= 90 THEN '2_Months_Ago'
    END AS timeframe, count(DISTINCT r._id) as receipt_count
FROM brands b
JOIN receipts r
ON r.rewardsReceiptItemList LIKE '%' || b.brandCode || '%'
JOIN users u
ON r.userID = u._id
WHERE b.brandCode IS NOT NULL
AND ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) <= 90
AND u.role = 'consumer'
GROUP BY 1,2
ORDER BY 2,3 DESC


'''

brands_sql = pd.read_sql(query, conn)
brands_sql.to_sql('brands_sql', conn, if_exists='replace')
display(brands_sql)
brands_sql.to_csv('brands_sql.csv')

,brandCode,timeframe,receipt_count
0,,2_Months_Ago,119
1,SARGENTO,2_Months_Ago,15
2,OSCAR MAYER,2_Months_Ago,9
3,AMP,2_Months_Ago,7
4,ONE,2_Months_Ago,6
...,...,...,...
135,SIERRA MIST,Previous_Month,1
136,STOVE TOP,Previous_Month,1
137,TIGI,Previous_Month,1
138,WYLER'S,Previous_Month,1


Analyzed overall transaction counts to build a broader context of consumer activity, as it could potentially help explain any shifts in brand ranking.

In [54]:
query = '''
SELECT  CASE WHEN ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) <= 30 THEN 'Current_Month'
   WHEN ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) > 30 AND ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) <= 60 THEN 'Previous_Month'
    WHEN ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) > 60 AND ABS(julianday('2021-03-08') - julianday(r.purchaseDate)) <= 90 THEN '2_Months_Ago'
    END AS timeframe, count(DISTINCT r._id) as receipt_count
FROM receipts r
JOIN users u
ON r.userID = u._id
WHERE u.role = 'consumer'
GROUP BY 1

'''
users_sql = pd.read_sql(query, conn)
users_sql.to_sql('users_sql', conn, if_exists='replace')
display(users_sql)
#users_sql.to_csv('users_sql.csv')

,timeframe,receipt_count
0,None,39
1,2_Months_Ago,119
2,Current_Month,12
3,Previous_Month,230


Final analysis can be found in the root folder of the github folder, with the summary available in the brand ranking summary in "Brand Ranking Summary.pdf" and the brand ranking data in "Brand Pivot.xlsx".